<a href="https://colab.research.google.com/github/Saikumarpagoti/FMML-MODULUS--IIIT-HYDERABAD/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [15]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [16]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-16-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1.The percentage of data allocated to the validation set can have a significant impact on the accuracy of the validation set, and it's important to strike the right balance. Let's explore how increasing and reducing the percentage of the validation set affects model evaluation:

Increasing the Percentage of the Validation Set:

Pros:
A larger validation set provides a better estimate of how well the model is likely to perform on unseen data because it reflects a more representative portion of the data distribution.
It helps reduce the variance in the estimate of model performance, leading to more stable and reliable evaluation results.
Cons:
With a larger validation set, you have fewer examples for training, which can lead to a smaller training dataset. This can be problematic if you already have limited training data, potentially resulting in underfitting.
Reducing the Percentage of the Validation Set:

Pros:
A smaller validation set allows for a larger training set, which can be beneficial when you have limited data. A larger training set may help the model learn more effectively.
It's computationally efficient because you're training on a larger portion of the data.
Cons:
A smaller validation set may not provide as reliable an estimate of model performance, as it may be more sensitive to random variations in the validation data. The estimate can have higher variance.
There's a risk of overfitting to the training data because you have less validation data to monitor the model's generalization.
In practice, the choice of the percentage of the validation set depends on factors such as the size of your dataset, the complexity of your model, and your computational resources. A common practice is to use a 70-80% training and 20-30% validation split when you have a reasonably sized dataset. This allows for a substantial training set while still providing a reliable estimate of model performance.

If you have a very large dataset, you can allocate a smaller percentage to the validation set because even a small portion of a large dataset can provide a representative sample. Conversely, if you have a very small dataset, you may need to allocate a larger percentage to the validation set to ensure a robust estimate of performance.

Ultimately, the choice of the validation set percentage should be guided by the need for reliable evaluation and the trade-off between having sufficient training data and a representative validation set. Cross-validation techniques can also be useful in situations where you want to make the most efficient use of your limited data while maintaining robust model evaluation.

2.The size of the train and validation sets can indeed affect how well you can predict the accuracy on the test set using the validation set. The primary factor at play is the representativeness of the validation set and how well it captures the underlying data distribution. Here's how the size of these sets can impact prediction accuracy:

Large Validation Set:

Pros:
A large validation set is more likely to be representative of the data distribution, especially when the dataset is substantial. It can provide a robust estimate of the model's performance.
Predicting test accuracy based on a large validation set tends to be more accurate because the validation set closely resembles the test set in terms of data characteristics.
Cons:
A larger validation set means a smaller training set, which can be problematic if you have limited training data. This might lead to underfitting or difficulty in training complex models.
Small Validation Set:

Pros:

A smaller validation set allows for a larger training set, which can be beneficial when you have limited data. More training data can lead to better model performance.
With a smaller validation set, you may have fewer examples to evaluate, making it computationally efficient.
Cons:

A small validation set may not be as representative of the data distribution, and the estimate of model performance may have higher variance. It may not generalize as well to the test set.
Predicting test accuracy based on a small validation set can be less reliable because the validation set may not capture the true variability of the test data.
In practice, the balance between the size of the training and validation sets depends on the specific problem, dataset size, and model complexity. It's crucial to strike a balance that allows you to both train a model effectively and reliably estimate its performance.

To mitigate the limitations of small validation sets, consider techniques like k-fold cross-validation. Cross-validation involves splitting the data into multiple folds, allowing you to train and validate the model multiple times while ensuring that each example has a chance to be in both the training and validation sets. This helps provide a more robust estimate of model performance, even with a small validation set.

3.The choice of the percentage to reserve for the validation set is dependent on several factors, including the size of your dataset, the complexity of your model, and your specific goals. There is no one-size-fits-all answer, but I can provide some general guidelines:

Common Split Ratios: In practice, a commonly used split ratio is 70-80% of the data for training and 20-30% for validation. This allows for a substantial training set while still providing a reasonably sized validation set for reliable performance estimation.

Small Datasets: If you have a relatively small dataset (e.g., a few hundred examples), you might consider allocating a larger percentage (e.g., 30%) to the validation set to ensure a more robust estimate of model performance. This is because smaller datasets are more prone to random fluctuations.

Large Datasets: When working with very large datasets, you can allocate a smaller percentage (e.g., 20%) to the validation set, as even a small portion of a large dataset can be quite representative.

Complex Models: If you are using a highly complex model that requires a lot of data to train effectively (e.g., deep neural networks with millions of parameters), you may want to allocate a larger portion to the training set to ensure the model has sufficient data to learn from.

Cross-Validation: In situations where you need a highly reliable estimate of model performance, especially when dealing with limited data, consider using cross-validation (e.g., k-fold cross-validation). Cross-validation divides the data into multiple folds, allowing each data point to be in both the training and validation sets at different times. This can help provide a more robust estimate of model performance.

Stratified Sampling: When splitting your data, consider using stratified sampling to ensure that the class distribution in both the training and validation sets is similar to the overall dataset. This is important for maintaining data balance.

Ultimately, the goal is to strike a balance between having enough data for training and having a representative validation set for reliable performance estimation. You may need to experiment with different split ratios and techniques like cross-validation to determine what works best for your specific problem and constraints. The key is to ensure that the validation set is representative enough to provide a meaningful estimate of how your model will perform on unseen

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1 Yes, averaging validation accuracy across multiple splits can indeed provide more consistent and reliable results when evaluating the performance of a machine learning model, especially in situations where the dataset is relatively small or where there is randomness involved, such as in cross-validation.

Here's why averaging validation accuracy across multiple splits can be beneficial:

Reduced Variance: Different random splits of the data can lead to variations in the validation accuracy. By averaging over multiple splits, you reduce the impact of this randomness and get a more stable estimate of the model's performance.

Better Generalization Estimate: Averaging over multiple splits helps you obtain a more robust estimate of how well your model is likely to generalize to unseen data. This is important because a single split may result in a dataset that is not representative of the overall distribution.

Identifying Overfitting: If a model performs exceptionally well on one specific split but poorly on others, it could be a sign of overfitting to the idiosyncrasies of that particular split. Averaging across splits helps to mitigate this by focusing on the model's average performance.

Improved Confidence Intervals: When you average validation accuracy over multiple splits, you can also compute confidence intervals to quantify the uncertainty in your performance estimate. This provides a more informative summary of the model's expected performance.

Common techniques for averaging validation accuracy include k-fold cross-validation and stratified sampling. In k-fold cross-validation, the dataset is divided into k subsets (folds), and the model is trained and evaluated k times, each time using a different fold as the validation set. The results are then averaged to obtain a final performance estimate.

2.Averaging validation accuracy across multiple splits can provide a more accurate estimate of a model's expected test accuracy, but it's important to clarify what is meant by "test accuracy" in this context.

In machine learning, there are typically two types of datasets used for evaluation:

Validation Dataset: This is a dataset that is used during the model development and training phase. It helps you make decisions about hyperparameters, model selection, and any other tuning. The validation dataset is used to estimate how well your model is likely to perform on unseen data.

Test Dataset: This is a separate, unseen dataset that is not used during model development. It is reserved for final model evaluation and provides an estimate of how well your model is expected to perform on completely new and unseen data.

Averaging validation accuracy across multiple splits helps provide a more accurate estimate of how well your model might perform on new, unseen data, which is akin to estimating test accuracy. However, it's crucial to understand that it's still an estimate, and there can be some degree of variation between the estimated performance and the actual test performance.

The reason for this is that the validation data, even when divided into multiple folds or splits, is still drawn from the same dataset as your training data. It doesn't fully capture the diversity and variability of completely new and unseen data that the test dataset represents. Therefore, while averaging validation accuracy gives you a more robust estimate of model performance, it may not perfectly predict how well your model will perform on the true test data.

To get the most accurate estimate of test accuracy, it's essential to use a separate, completely independent test dataset that the model has never seen during training or validation. This test dataset should ideally come from the same distribution as the data you expect your model to encounter in the real world. The test accuracy on this dataset provides the most accurate estimate of how your model will perform in practice.

3.The number of iterations, often referred to as the number of training epochs, can have a significant effect on the estimate of a model's performance during training and validation. In general, increasing the number of iterations can lead to a better estimate of a model's performance, but it's essential to strike a balance to avoid overfitting and computational inefficiency.

Here's how the number of iterations can affect the estimate of a model's performance:

Underfitting: If you have too few iterations (epochs), the model may not have enough opportunities to learn from the training data, and it may underfit the data. In this case, both the training and validation performance may be suboptimal.

Convergence: As you increase the number of iterations, the model is more likely to converge to a better solution, which can result in improved performance on both the training and validation datasets. It allows the model to capture complex patterns in the data.

Overfitting: However, if you increase the number of iterations excessively, the model may start to memorize the training data (overfit), leading to excellent training performance but poor validation performance. This is because the model becomes too specialized in the training data and does not generalize well to new, unseen data.

The key is to find the right balance in the number of iterations, often through experimentation and model validation techniques like early stopping. Early stopping involves monitoring the validation performance during training and stopping when it starts to degrade, indicating that the model is overfitting.

4.Increasing the number of iterations (training epochs) can help to some extent when dealing with a very small training dataset, but it's not a complete solution, and it comes with certain limitations and risks:

Overfitting: When you have a very small training dataset, increasing the number of iterations can lead to overfitting. The model might start memorizing the limited data it has rather than learning meaningful patterns. This can result in excellent training performance but poor generalization to new data.

Limited Generalization: Even with more iterations, a model trained on a tiny dataset may struggle to generalize well to unseen data because it hasn't seen a representative sample of the underlying data distribution. More iterations may not fully address this limitation.

Computational Cost: Training for more iterations can be computationally expensive, especially for complex models or when working with limited computational resources.

To address the challenges of small training and validation datasets, it's often better to consider alternative strategies:

Data Augmentation: If your training dataset is small, you can apply data augmentation techniques to artificially increase the effective size of your training dataset. Data augmentation involves creating new training examples by applying transformations (e.g., rotations, flips, crops) to the existing data.

Transfer Learning: Consider using pre-trained models as a starting point. Transfer learning allows you to leverage knowledge learned from a larger and more diverse dataset, and then fine-tune the model on your smaller dataset. This can be highly effective when you have limited data.

Regularization: Apply regularization techniques like dropout or L1/L2 regularization to help prevent overfitting when training on a small dataset.

Ensemble Methods: Combine multiple models, each trained on different subsets of your small dataset, to reduce the risk of overfitting and improve generalization.

Stratified Sampling: When splitting your data into training and validation sets, use stratified sampling to ensure that both sets maintain the same class distribution as the original data. This can help ensure that the validation dataset is representative.

Cross-Validation: Use cross-validation with multiple folds to assess the model's performance more reliably, even with a small dataset.